In [1]:
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = '1'  # Set to the GPU you want to use, or '0' for the first GPU
import torch
num_gpus = torch.cuda.device_count()
print("number of GPUs available:", torch.cuda.device_count())
for i in range(num_gpus):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

number of GPUs available: 1
GPU 0: NVIDIA L20


In [2]:
from atlas_rag.retriever import NvEmbed
from transformers import AutoModel
# Load the SentenceTransformer model
encoder_model_name = "nvidia/NV-Embed-v2"
sentence_model = AutoModel.from_pretrained(encoder_model_name, trust_remote_code=True, device_map="auto")
sentence_encoder = NvEmbed(sentence_model)

/home/httsangaj/miniconda3/envs/faiss-gpu/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/httsangaj/projects/AutoSchemaKG/atlas_rag/utils/json_2_csv.py:22: SyntaxWarning: invalid escape sequence '\e'
  new_text = text.replace("\n", " ").replace("\r", " ").replace("\t", " ").replace("\v", " ").replace("\f", " ").replace("\b", " ").replace("\a", " ").replace("\e", " ").replace(";", ",")
/home/httsangaj/projects/AutoSchemaKG/atlas_rag/kg_construction/concept_generation.py:100: SyntaxWarning: invalid escape sequence '\e'
  new_text = text.replace("\n", " ").replace("\r", " ").replace("\t", " ").replace("\v", " ").replace("\f", " ").replace("\b", " ").replace("\a", " ").replace("\e", " ").replace(";", ",")
Loading checkpoint shards: 100%|██████████| 4/4 [00:45<00:00, 11.37s/it]


In [3]:
from openai import OpenAI
from atlas_rag.reader import LLMGenerator
from configparser import ConfigParser
# Load OpenRouter API key from config file
config = ConfigParser()
config.read('config.ini')
reader_model_name = "meta-llama/Llama-3.3-70B-Instruct"
client = OpenAI(
  base_url="https://api.deepinfra.com/v1/openai",
  api_key=config['settings']['DEEPINFRA_API_KEY'],
)
llm_generator = LLMGenerator(client=client, model_name=reader_model_name)

In [4]:
from atlas_rag import create_embeddings_and_index
keyword = 'musique'
working_directory = f'/data/httsangaj/atomic-rag/8b'
data = create_embeddings_and_index(
    sentence_encoder=sentence_encoder,
    model_name = 'nvidia/NV-Embed-v2',
    working_directory=working_directory,
    keyword=keyword,
    include_concept=True,
    include_events=True,
    normalize_embeddings= True,
    text_batch_size=64,
    node_and_edge_batch_size=64,
)

Using encoder model: NV-Embed-v2
Loading graph from /data/httsangaj/atomic-rag/8b/kg_graphml/musique_graph.graphml


100%|██████████| 262675/262675 [00:00<00:00, 1754864.14it/s]
955769it [00:00, 3415846.10it/s]


Text embeddings already computed.
Graph embeddings computed
Node and edge embeddings already computed.


In [5]:
from atlas_rag.evaluation import BenchMarkConfig
benchmark_config = BenchMarkConfig(
    dataset_name= 'musique',
    question_file= "benchmark_data/musique.json",
    include_concept=True,
    include_events=True,
    reader_model_name=reader_model_name,
    encoder_model_name=encoder_model_name,
    number_of_samples=-1, # -1 for all samples
)

In [10]:
from atlas_rag import setup_logger
logger = setup_logger(benchmark_config)

In [11]:
# Initialize desired RAG method for benchmarking
from atlas_rag.retriever import HippoRAG2Retriever
hipporag2_retriever = HippoRAG2Retriever(
    llm_generator=llm_generator,
    sentence_encoder=sentence_encoder,
    data = data,
    logger=logger
)

100%|██████████| 262675/262675 [00:00<00:00, 546845.71it/s]


In [ ]:
# start benchmarking
from atlas_rag.evaluation import RAGBenchmark
benchmark = RAGBenchmark(config=benchmark_config, logger=logger)
benchmark.run([hipporag2_retriever], llm_generator=llm_generator)

Data loaded from benchmark_data/musique.json


  0%|          | 0/1000 [00:00<?, ?it/s]